In [2]:
import altair as alt
import os
import pandas as pd
import re
from transformers import T5Tokenizer

In [3]:
DIR = os.path.abspath('')
ANALYSIS_DIR = os.path.join(os.path.dirname(DIR), 'data', 'rouge_analysis')
with open(os.path.join(ANALYSIS_DIR, 'articles'), 'r') as f:
    articles = f.readlines()
ext_df, abs_df, hybrid_df = (
    pd.read_csv(
        os.path.join(ANALYSIS_DIR, f'{model}_rouge.csv'),
        index_col=0,
    )
    for model in ('extractive', 'abstractive', 'hybrid')
)

In [6]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [11]:
data = pd.DataFrame({
    'doc_id': range(len(articles)),
    'char_len': map(len, articles),
    'word_len': (len(re.split('\s+', x)) for x in articles),
    'token_len': (len(tokenizer.encode(x)) for x in articles),
})

Token indices sequence length is longer than the specified maximum sequence length for this model (533 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (539 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (588 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (733 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (697 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

Token indices sequence length is longer than the specified maximum sequence length for this model (2499 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1890 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1929 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2291 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1016 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length fo

Token indices sequence length is longer than the specified maximum sequence length for this model (978 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (995 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1484 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (850 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1462 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for t

Token indices sequence length is longer than the specified maximum sequence length for this model (845 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (949 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1419 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (863 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (639 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Token indices sequence length is longer than the specified maximum sequence length for this model (2341 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (617 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1534 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1977 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (860 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

...

Token indices sequence length is longer than the specified maximum sequence length for this model (1665 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (683 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (935 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1512 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (3005 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

Token indices sequence length is longer than the specified maximum sequence length for this model (957 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1423 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1028 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1585 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1702 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for

Token indices sequence length is longer than the specified maximum sequence length for this model (1198 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (781 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (825 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1123 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1912 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

Token indices sequence length is longer than the specified maximum sequence length for this model (848 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1499 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (951 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1207 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1575 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for 

Token indices sequence length is longer than the specified maximum sequence length for this model (1123 > 512). Running this sequence through the model will result in indexing errors


In [25]:
data['token_bin'] = pd.qcut(
    data['token_len'],
    q=10,
    labels=range(10),
)

In [26]:
model_dfs = (
    ('extractive', ext_df),
    ('abstractive', abs_df),
    ('hybrid', hybrid_df),
)
for label, df in model_dfs:
    df['label'] = label

In [27]:
full_data = pd.concat(
    (data.merge(df, on='doc_id', how='left') for _, df in model_dfs),
    axis=0,
)

In [19]:
full_data.columns

Index(['doc_id', 'char_len', 'word_len', 'token_len', 'rouge_1_recall',
       'rouge_1_recall_cb', 'rouge_1_recall_ce', 'rouge_1_precision',
       'rouge_1_precision_cb', 'rouge_1_precision_ce', 'rouge_1_f_score',
       'rouge_1_f_score_cb', 'rouge_1_f_score_ce', 'rouge_2_recall',
       'rouge_2_recall_cb', 'rouge_2_recall_ce', 'rouge_2_precision',
       'rouge_2_precision_cb', 'rouge_2_precision_ce', 'rouge_2_f_score',
       'rouge_2_f_score_cb', 'rouge_2_f_score_ce', 'rouge_l_recall',
       'rouge_l_recall_cb', 'rouge_l_recall_ce', 'rouge_l_precision',
       'rouge_l_precision_cb', 'rouge_l_precision_ce', 'rouge_l_f_score',
       'rouge_l_f_score_cb', 'rouge_l_f_score_ce', 'label'],
      dtype='object')

In [22]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [24]:
alt.Chart(data).mark_bar().encode(
    alt.X('token_len:Q', bin=True),
    y='count()',
)

alt.Chart(...)

In [35]:
binned_data = full_data.groupby(['token_bin', 'label']).mean().reset_index()

In [36]:
alt.Chart(binned_data).mark_line().encode(
    x='token_len',
    y='rouge_1_f_score',
    color='label',
)

alt.Chart(...)

In [37]:
alt.Chart(binned_data).mark_line().encode(
    x='token_len',
    y='rouge_2_f_score',
    color='label',
)

alt.Chart(...)

In [38]:
alt.Chart(binned_data).mark_line().encode(
    x='token_len',
    y='rouge_l_f_score',
    color='label',
)

alt.Chart(...)

In [40]:
alt.Chart(binned_data).mark_line().encode(
    x='token_len',
    y='rouge_1_recall',
    color='label',
)

alt.Chart(...)

In [41]:
alt.Chart(binned_data).mark_line().encode(
    x='token_len',
    y='rouge_2_recall',
    color='label',
)

alt.Chart(...)

In [42]:
alt.Chart(binned_data).mark_line().encode(
    x='token_len',
    y='rouge_l_recall',
    color='label',
)

alt.Chart(...)